In [1]:
import nltk

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [4]:
# Libraries needed for tensorflow processing
import tensorflow as tf
import numpy as np
import random
import json

In [ ]:
from google.colab import files
files.upload()

In [6]:
# import our chat - bot intents file
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [7]:
words = []
classes = []
documents = []
ignore = ['?']

# loop through each sentence in the intent's pattern
for intent in intents['intents']:
  for pattern in intent['patterns']:
    # tokenize each and every word in the sentence
    w = nltk.word_tokenize(pattern)
    # add word to the words list
    words.extend(w)
    # add words(s) to documents
    documents.append((w, intent['tag']))
    # add tags to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [8]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

In [9]:
# remove duplicate classes
classes = sorted(list(set(classes)))

In [11]:
# Create training data
training = [] # X -- data
output = [] # Y -- label

# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
  # initialize bag of words
  bag = []
  # list of tokenized words for the pattern
  pattern_words = doc[0]
  # stemming each word
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  # create bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

  # output is '1' for current tag and '0' for rest of other tags
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row])

# shffling feature and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:, 0])
train_y = list(training[:, 1])

<ipython-input-11-e0d1558a1b5a>:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, input_shape=(len(train_x[0]),))) # we are taking 10 neuron in first param
model.add(tf.keras.layers.Dense(10)) #
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax')) # More than two classes is multi-classification . For Multi-classification we have to use softmax
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=8, verbose=1) # fit use for train the model
model.save("model.pkl")

Epoch 1/100
16/16 [==============================] - 2s 4ms/step - loss: 3.7314 - accuracy: 0.0909
Epoch 2/100
16/16 [==============================] - 0s 4ms/step - loss: 3.6918 - accuracy: 0.1405
Epoch 3/100
16/16 [==============================] - 0s 3ms/step - loss: 3.6555 - accuracy: 0.1653
Epoch 4/100
16/16 [==============================] - 0s 3ms/step - loss: 3.6178 - accuracy: 0.1818
Epoch 5/100
16/16 [==============================] - 0s 3ms/step - loss: 3.5789 - accuracy: 0.1983
Epoch 6/100
16/16 [==============================] - 0s 4ms/step - loss: 3.5341 - accuracy: 0.2149
Epoch 7/100
16/16 [==============================] - 0s 3ms/step - loss: 3.4879 - accuracy: 0.2397
Epoch 8/100
16/16 [==============================] - 0s 4ms/step - loss: 3.4336 - accuracy: 0.2645
Epoch 9/100
16/16 [==============================] - 0s 2ms/step - loss: 3.3746 - accuracy: 0.2645
Epoch 10/100
16/16 [==============================] - 0s 2ms/step - loss: 3.3097 - accuracy: 0.2810
Epoch 11/

In [14]:
import pickle
pickle.dump( {'words': words, 'classes': classes}, open("training_data", "wb"))


In [15]:
from tensorflow.keras.models import load_model
model = load_model("model.pkl")

In [16]:
# restoring all the data structures
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']

In [17]:
with open("intents.json") as json_data:
  intents = json.load(json_data)

In [18]:
def clean_up_sentence(sentence):
  # tokenizing the pattern
  sentence_words = nltk.word_tokenize(sentence)
  # stemming each word
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
  # tokenizing the pattern
  sentence_words = clean_up_sentence(sentence)
  # generating bag of words
  bag = [0]*len(words)
  for s in sentence_words:
    for i, w in enumerate(words):
      if w == s:
        bag[i] = 1
  bag=np.array(bag)
  return (bag)


In [19]:
ERROR_THRESHOLD = 0.30

def classify(sentence):
  # generate probabilities from the model
  bag = bow(sentence, words)
  results = model.predict(np.array([bag]))
  # filter out predictions below a threshold
  results = [[i,r] for i,r in enumerate(results[0]) if r > ERROR_THRESHOLD]
  # sort by strength of probability
  results.sort(key=lambda x:x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  # return tuple of intent and probablity
  return return_list

In [23]:
def response(sentence):
  results = classify(sentence)
  # if we have a classification then find the matching intent tag
  if results:
    # loop as long as there are matches to process
    count = 0
    while results:
      for i in intents['intents']:
        # find a tag matcing the first result
        if i['tag'] == results[0][0]:
          # a random response from the intent
          return print(random.choice(i['responses']))
        results.pop(0)

In [24]:
response('Hi how are you')

1/1 [==============================] - 0s 32ms/step
Hello, thanks for asking
